# Crime map

This notebooks show geospatial locations of crimes in St. Louis, MO between the years 2008-2016.  Below, the city neighborhoods are outlined (using geojson boundaries extrated from a neighborhood boundary shapefile), and the locations of all homocides are shown.  The colors correspond to the temperature at the time of the crime; red is a warmer temperature, while black/purple is a colder temperature

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
from IPython.display import HTML

%matplotlib notebook

Here we create a dataframe of exclusively homocides.

In [26]:
crimedf = pd.read_pickle('cleaned_data/crimedf.pkl')
crimedf['Homocide'] = 0
crimedf.loc[crimedf['Crime'].str[0:2] == '01', 'Homocide'] = 1
crimedfhomocide = crimedf[(crimedf['Homocide']==1)]
crimedfhomocide = crimedfhomocide.reset_index().drop(['index'],1)

In [28]:
StLouis = json.load(open('StL.json'))
m = folium.Map(location=[38.65, -90.25], tiles='CartoDB positron', zoom_start=11, width='60%')
highlight_function = lambda x: {'fillColor': 'black', 'color': 'black', 'weight': 4, 'fillOpacity': 0}
style_function = lambda feature: {'fillColor': 'black', 'color': 'black', 'weight' : 1, 'fillOpacity' : 0.025}
folium.GeoJson(StLouis, smooth_factor = 1, style_function = style_function, highlight_function = highlight_function).add_to(m)
for ind in range(0,crimedfhomocide.shape[0],1):
    c = '#%02x%02x%02x' % tuple(np.round(256*np.array(plt.cm.gnuplot(crimedfhomocide.loc[ind, 'dry_temp']/120)[0:3])).astype(int))
    folium.CircleMarker(location=[crimedfhomocide.loc[ind,'YCoord'], crimedfhomocide.loc[ind,'XCoord']], radius=4, popup=str(crimedfhomocide.loc[ind,'Date']) + crimedfhomocide.loc[ind,'Description'], color='#00', fill_color=c, fill_opacity=0.5,fill=True).add_to(m)

m.save('StL_homocide_map.html')

In [29]:
HTML('<iframe src=StL_homocide_map.html width=1600 height=800></iframe>')